# Feature Selection & Feature Importances

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error

![baby penguin gif from Giphy](https://media.giphy.com/media/RiJuDMqd6vDgfPrZN2/giphy.gif)

Let's hang out with some penguins, loading up a dataset straight from seaborn:

In [ ]:
data = sns.load_dataset('penguins')

In [ ]:
data.head()

In [ ]:
data.info()

Let's clean up this dataset - two rows have quite a few null values, and 11 total do not have a value for `sex`, so let's drop rows where any data is null:

In [ ]:
# Drop nulls here
data = data.dropna(how='any')

In [ ]:
# Sanity check
data.isna().sum()

In [ ]:
data.info()

### Encoding Our Data

In [ ]:
data[['species','island','sex']].describe()

In order to use the `gender`, `species` or `island` data we need to render those strings as numbers - since there are only 2-3 unique values per column, let's simply one-hot-encode those columns (aka turn the columns into a series of binary indicators).

Using Pandas' `get_dummies` : https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.get_dummies.html

In [ ]:
# One hot encode our three 'object' columns
data_num = pd.get_dummies(data, columns=['species', 'island', 'sex'],
                          drop_first=True)

In [ ]:
data_num.head()

You'll note that Pandas' `get_dummies` drops the first option, which means one of the species and one of the islands won't be obvious in our features. A thing to keep in mind!

In [ ]:
sns.heatmap(data_num.corr().abs())
plt.show()

In [ ]:
sns.pairplot(data_num)

The goal of this is to predict body mass, `body_mass_g`, so let's define our X and y and perform a train/test split:

In [ ]:
# Define X and y
X_cols = [c for c in data_num.columns.to_list() if c not in ['body_mass_g']]

X = data_num[X_cols]
y = data_num.body_mass_g

In [ ]:
# Perform a train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
len(X_test) + len(X_train) == len(X)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
# Instantiate a scaler
scaler = StandardScaler()

In [ ]:
# Train our scaler on training data, then fit to testing
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Baseline Linear Regression Model

In [ ]:
# Instantiate a linear regression model
lr = LinearRegression()

In [ ]:
# Fit our model on our scaled data
lr.fit(X_train_scaled, y_train)

In [ ]:
# Evaluate
y_train_pred = lr.predict(X_train_scaled)
y_test_pred = lr.predict(X_test_scaled)

print("Training Scores:")
print(f"R2: {r2_score(y_train, y_train_pred)}")
print(f"Mean Absolute Error: {mean_absolute_error(y_train, y_train_pred)}")
print("---")
print("Testing Scores:")
print(f"R2: {r2_score(y_test, y_test_pred)}")
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_test_pred)}")

## Feature Importance through Coefficients

Because we've scaled our data, we can explore our coefficients to see which are having more of an impact on our model.

Note! This, or using p-values from a statsmodels model, is all I expect you to do in this project - anything else we talk about today is completely optional and going above and beyond!

In [ ]:
# look at the coefficients with the names of each col
dict(zip(X.columns, lr.coef_))

In [ ]:
# let's also look at the intercept
lr.intercept_

### [ELI5](https://eli5.readthedocs.io/en/latest/index.html)

ELI5, short for 'explain like I'm 5', is a library specifically designed to help explore feature importances. It'll help you visualize exactly what we just did above!

In [ ]:
import eli5

In [ ]:
# don't have eli5? uncomment the below code
# pip install eli5

In [ ]:
eli5.show_weights(lr, feature_names=list(X.columns))

### So what?

You can use feature importances, or an exploration of statistical significance through statsmodels' provided p-values, to decide which features to keep and which ones to drop in your next iteration of your model! Especially if you OHE a lot of columns, you might use this technique to grab only useful indicators rather than keeping every feature you build in your model.

Will dropping seemingly 'low' features in terms of feature importance always help improve your scores? Nope! Will each way of understanding which feature is 'important' always have the same results? Nope! But try different techniques, see what you think and what helps your model, then iterate.

In terms of interpretation, you may want to get to a final model in terms of what features are used and how you process your data - then build a version of that final model on data that isn't scaled so you can see exactly how each unit of each feature directly impacts your target!

In [ ]:
lr_ns = LinearRegression()

lr_ns.fit(X_train, y_train)

# Evaluate
y_train_pred = lr_ns.predict(X_train)
y_test_pred = lr_ns.predict(X_test)

# these scores should be exactly the same
print("Training Scores:")
print(f"R2: {r2_score(y_train, y_train_pred)}")
print(f"Mean Absolute Error: {mean_absolute_error(y_train, y_train_pred)}")
print("---")
print("Testing Scores:")
print(f"R2: {r2_score(y_test, y_test_pred)}")
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_test_pred)}")

In [ ]:
# BUT the feature importances will be totally different
eli5.show_weights(lr_ns, feature_names=list(X.columns))

Rather than interpret anything about this (you can NOT interpret relative feature importances based on unscaled coefficients!!), you can use these coefficients now to discuss how each unit affects the target.

Example: each additional mm of flipper length adds 17.472g of body mass, being a Gentoo penguin adds 810.346g of body mass, etc.

-----

## Above and Beyond: Regularization Terms

Suppose I have split my data into training and testing sets. Do I want my model to fit my training data _exactly_?

Often, the answer is actually 'NO!', because doing that will lead to an overfit model.

Overfitting is generally a result of high variance. High variance can be caused by:

- having irrelevant or too many predictors
- multicollinearity
- large coefficients

The first problem is about picking up on noise rather than signal.

The second problem is about having a least-squares estimate that is highly sensitive to random error.

The third is about having highly sensitive predictors.

Regularization works by introducing a factor into our model designed to enforce the stricture that the coefficients stay small, by penalizing the ones that get too large.

That is, we'll alter our loss function so that the goal now is not merely to minimize the difference between actual values and our model's predicted values. Rather, we'll add in a term to our loss function that represents the sizes of the coefficients.

There are two popular ways of doing this:

Lasso ("L1"): Minimize $\large\Sigma^{n_{obs.}}_{i=1}[(y_i - \Sigma^{n_{feat.}}_{j=0}\beta_j\times x_{ij})^2 + \lambda\Sigma^{n_{feat.}}_{j=0}|\beta_j|]$
<br/> <br/>

Ridge ("L2"): Minimize $\large\Sigma^{n_{obs.}}_{i=1}[(y_i - \Sigma^{n_{feat.}}_{j=0}\beta_j\times x_{ij})^2 + \lambda\Sigma^{n_{feat.}}_{j=0}\beta^2_j]$

**$\rightarrow$ Don't let these formulas be intimidating. The first term in each of these (the sum of squares) is the same, and is just the familiar loss function that we've always used. What distinguishes the Lasso Regression from the Ridge Regression is only the extra term on the right. The Lasso uses the absolute values of the coefficients, while the Ridge uses the squares of the coefficients.**

For a given value of $\lambda$, the Ridge makes for a gentler reining in of runaway coefficients. The Lasso will more quickly reduce the contribution of individual predictors down to insignificance.

For a nice discussion of these methods in Python, see https://towardsdatascience.com/ridge-and-lasso-regression-a-complete-guide-with-python-scikit-learn-e20e34bcbf0b.

**TL;DR:**

- L1 Regularization (LASSO) is good for feature selection

- L2 Regularization (Ridge) is good for reducing the impact of multicollinear features

(also, you can do both at the same time with a technique called [Elastic Net](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html))

### L1 Norm: [LASSO](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)

(Least Absolute Shrinkage and Selection Operator)

In [ ]:
# Instantiate a lasso regression model
lasso = Lasso(alpha=.5)

In [ ]:
# Fit your new L1 model -  on the scaled data
lasso.fit(X_train_scaled, y_train)

In [ ]:
# Evaluate
y_train_pred_l1 = lasso.predict(X_train_scaled)
y_test_pred_l1 = lasso.predict(X_test_scaled)

print("Training Scores:")
print(f"R2: {r2_score(y_train, y_train_pred_l1)}")
print(f"Mean Absolute Error: {mean_absolute_error(y_train, y_train_pred_l1)}")
print("---")
print("Testing Scores:")
print(f"R2: {r2_score(y_test, y_test_pred_l1)}")
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_test_pred_l1)}")

Remember - what's the benefit of using LASSO?

In [ ]:
data_num.columns

In [ ]:
print("Unpenalized Linear Regression Coefficients are:{}".format(lr.coef_))
print("Unpenalized Linear Regression Intercept:{}".format(lr.intercept_))
print("---")
print("Lasso Regression Coefficients are:{}".format(lasso.coef_))
print("Lasso Linear Regression Intercept:{}".format(lasso.intercept_))

### L2 Norm: [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)

In [ ]:
# Instantiate a lasso regression model
ridge = Ridge(alpha=.5)

In [ ]:
# Fit your new L2 model -  on the scaled data
ridge.fit(X_train_scaled, y_train)

In [ ]:
# Evaluate
y_train_pred_l2 = ridge.predict(X_train_scaled)
y_test_pred_l2 = ridge.predict(X_test_scaled)

print("Training Scores:")
print(f"R2: {r2_score(y_train, y_train_pred_l2)}")
print(f"Mean Absolute Error: {mean_absolute_error(y_train, y_train_pred_l2)}")
print("---")
print("Testing Scores:")
print(f"R2: {r2_score(y_test, y_test_pred_l2)}")
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_test_pred_l2)}")

In [ ]:
print("Unpenalized Linear Regression Coefficients are:{}".format(lr.coef_))
print("Unpenalized Linear Regression Intercept:{}".format(lr.intercept_))
print("---")
print("Ridge Regression Coefficients are:{}".format(ridge.coef_))
print("Ridge Linear Regression Intercept:{}".format(ridge.intercept_))

All together:

In [ ]:
coef_dict = {}
for loc, col in enumerate(data_num.columns):
    coef_dict[col] = {"Unpenalized": lr.coef_[loc-1],
                      "LASSO": lasso.coef_[loc-1],
                      "Ridge": ridge.coef_[loc-1]}
pd.DataFrame.from_dict(coef_dict)

### Alpha Levels??

We started with the **hyperparameter** alpha set to `0.5` for both our LASSO and Ridge Models: now let's play around with it!

In [ ]:
alphas = [.25, .5, 1, 10, 50, 100]

In [ ]:
for a in alphas:
    lasso_testing = Lasso(alpha = a)
    lasso_testing.fit(X_train_scaled, y_train)
    y_train_pred_testing = lasso_testing.predict(X_train_scaled)
    y_test_pred_testing = lasso_testing.predict(X_test_scaled)

    print(f"Training Scores at alpha: {a}")
    print(f"R2: {r2_score(y_train, y_train_pred_testing)}")
    print(f"Mean Absolute Error: {mean_absolute_error(y_train, y_train_pred_testing)}")
    print("---")
    print(f"Testing Scores at alpha: {a}")
    print(f"R2: {r2_score(y_test, y_test_pred_testing)}")
    print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_test_pred_testing)}")
    print("---")
    print(f"Coefficients at alpha: {a} are: {lasso_testing.coef_}")
    print(f"Intercept at alpha: {a} is:{lasso_testing.intercept_}")
    print("*" * 20)

In [ ]:
for a in alphas:
    ridge_testing = Ridge(alpha = a)
    ridge_testing.fit(X_train_scaled, y_train)
    y_train_pred_testing = ridge_testing.predict(X_train_scaled)
    y_test_pred_testing = ridge_testing.predict(X_test_scaled)

    print(f"Training Scores at alpha: {a}")
    print(f"R2: {r2_score(y_train, y_train_pred_testing)}")
    print(f"Mean Absolute Error: {mean_absolute_error(y_train, y_train_pred_testing)}")
    print("---")
    print(f"Testing Scores at alpha: {a}")
    print(f"R2: {r2_score(y_test, y_test_pred_testing)}")
    print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_test_pred_testing)}")
    print("---")
    print(f"Coefficients at alpha: {a} are: {ridge_testing.coef_}")
    print(f"Intercept at alpha: {a} is:{ridge_testing.intercept_}")
    print("*" * 20)

### Resources

- [Stats course resource from Penn State](https://online.stat.psu.edu/stat508/lesson/5), going into detail about Regression Shrinkage Methods - aka regularization. This is pretty technical, and the code is in R, but goes into good detail about the motivation of why we do this and how this works.

- Kaggle has a course on Machine Learning interpretability, with a section on [Permutation Importance](https://www.kaggle.com/dansbecker/permutation-importance) if you'd like to explore that